## Example 1. Programmatically obtaining a personal API access token for accessing GitHub's API

In [3]:
import requests
from getpass import getpass
import json

username = 'projjal1' # Your GitHub username
password = 'b8fbdfbae201f7b94e2df20cb9155dc0df1c1d7a' # Your GitHub password

# Note that credentials will be transmitted over a secure SSL connection
url = 'https://api.github.com/authorizations'
note = 'Mining the Social Web, 2nd Ed.'
post_data = {'scopes':['repo'],'note': note }

response = requests.post(
    url,
    auth = (username, password),
    data = json.dumps(post_data),
    )   

print ("API response:", response.text)
print()
print ("Your OAuth token is", response.json())

# Go to https://github.com/settings/applications to revoke this token

API response: {"message":"Not Found","documentation_url":"https://docs.github.com/rest"}

Your OAuth token is {'message': 'Not Found', 'documentation_url': 'https://docs.github.com/rest'}


## Example 2. Making direct HTTP requests to GitHub's API

In [4]:
import json
import requests

# An unauthenticated request that doesn't contain an ?access_token=xxx query string
url = "https://api.github.com/repos/ptwobrussell/Mining-the-Social-Web/stargazers"
response = requests.get(url)

# Display one stargazer

print (json.dumps(response.json()[0], indent=1))
print()

# Display headers
for (k,v) in response.headers.items():
    print (k, "=>", v)

{
 "login": "rdempsey",
 "id": 224,
 "node_id": "MDQ6VXNlcjIyNA==",
 "avatar_url": "https://avatars2.githubusercontent.com/u/224?v=4",
 "gravatar_id": "",
 "url": "https://api.github.com/users/rdempsey",
 "html_url": "https://github.com/rdempsey",
 "followers_url": "https://api.github.com/users/rdempsey/followers",
 "following_url": "https://api.github.com/users/rdempsey/following{/other_user}",
 "gists_url": "https://api.github.com/users/rdempsey/gists{/gist_id}",
 "starred_url": "https://api.github.com/users/rdempsey/starred{/owner}{/repo}",
 "subscriptions_url": "https://api.github.com/users/rdempsey/subscriptions",
 "organizations_url": "https://api.github.com/users/rdempsey/orgs",
 "repos_url": "https://api.github.com/users/rdempsey/repos",
 "events_url": "https://api.github.com/users/rdempsey/events{/privacy}",
 "received_events_url": "https://api.github.com/users/rdempsey/received_events",
 "type": "User",
 "site_admin": false
}

date => Tue, 17 Nov 2020 06:23:00 GMT
content-typ

## Example 3. Using PyGithub to query for stargazers of a particular repository

In [5]:
from github import Github

# XXX: Specify your own access token here

ACCESS_TOKEN = 'b8fbdfbae201f7b94e2df20cb9155dc0df1c1d7a'

# Specify a username and repository of interest for that user.

USER = 'ptwobrussell'
REPO = 'Mining-the-Social-Web'

client = Github(ACCESS_TOKEN, per_page=100)
user = client.get_user(USER)
repo = user.get_repo(REPO)

# Get a list of people who have bookmarked the repo.
# Since you'll get a lazy iterator back, you have to traverse
# it if you want to get the total number of stargazers.

stargazers = [ s for s in repo.get_stargazers() ]
print ("Number of stargazers", len(stargazers))

Number of stargazers 1196


In [15]:
#Check if you are stargazer
for sg in stargazers:
    if username in sg.login:
        print("Found")

Found


## Example 4. Constructing a trivial property graph

In [10]:
import networkx as nx

# Create a directed graph

g = nx.DiGraph()

# Add an edge to the directed graph from X to Y

g.add_edge('X', 'Y')

# Print some statistics about the graph

print (nx.info(g))
print()

# Get the nodes and edges from the graph

print ("Nodes:", g.nodes())
print ("Edges:", g.edges())
print()

# Get node properties

print ("X props:", g.node['X'])
print ("Y props:", g.node['Y'])

# Get edge properties

print ("X=>Y props:", g['X']['Y'])
print()

# Update a node property

g.node['X'].update({'prop1' : 'value1'})
print ("X props:", g.node['X'])
print()

# Update an edge property

g['X']['Y'].update({'label' : 'label1'})
print ("X=>Y props:", g['X']['Y'])

Name: 
Type: DiGraph
Number of nodes: 2
Number of edges: 1
Average in degree:   0.5000
Average out degree:   0.5000

Nodes: ['X', 'Y']
Edges: [('X', 'Y')]

X props: {}
Y props: {}
X=>Y props: {}

X props: {'prop1': 'value1'}

X=>Y props: {'label': 'label1'}


## Example 5. Constructing an ego graph of a repository and its stargazers

In [11]:
# Expand the initial graph with (interest) edges pointing each direction for 
# additional people interested. Take care to ensure that user and repo nodes 
# do not collide by appending their type.

import networkx as nx

g = nx.DiGraph()
g.add_node(repo.name + '(repo)', type='repo', lang=repo.language, owner=user.login)

for sg in stargazers:
    g.add_node(sg.login + '(user)', type='user')
    g.add_edge(sg.login + '(user)', repo.name + '(repo)', type='gazes')

## Example 6. Introducing some handy graph operations

In [16]:
# Poke around in the current graph to get a better feel for how NetworkX works

print (nx.info(g))
print()
print (g.node['Mining-the-Social-Web(repo)'])
print (g.node['ptwobrussell(user)'])
print()
print (g['ptwobrussell(user)']['Mining-the-Social-Web(repo)'])
# The next line would throw a KeyError since no such edge exists:
# print g['Mining-the-Social-Web(repo)']['ptwobrussell(user)']
print()
print (g['ptwobrussell(user)'])
print (g['Mining-the-Social-Web(repo)'])
print()
print (g.in_edges(['ptwobrussell(user)']))
print (g.out_edges(['ptwobrussell(user)']))
print()
print (g.in_edges(['Mining-the-Social-Web(repo)']))
print (g.out_edges(['Mining-the-Social-Web(repo)']))

Name: 
Type: DiGraph
Number of nodes: 1197
Number of edges: 1196
Average in degree:   0.9992
Average out degree:   0.9992

{'type': 'repo', 'lang': 'JavaScript', 'owner': 'ptwobrussell'}
{'type': 'user'}

{'type': 'gazes'}

{'Mining-the-Social-Web(repo)': {'type': 'gazes'}}
{}

[]
[('ptwobrussell(user)', 'Mining-the-Social-Web(repo)')]

[('rdempsey(user)', 'Mining-the-Social-Web(repo)'), ('frac(user)', 'Mining-the-Social-Web(repo)'), ('prb(user)', 'Mining-the-Social-Web(repo)'), ('mcroydon(user)', 'Mining-the-Social-Web(repo)'), ('batasrki(user)', 'Mining-the-Social-Web(repo)'), ('twleung(user)', 'Mining-the-Social-Web(repo)'), ('kevinchiu(user)', 'Mining-the-Social-Web(repo)'), ('nikolay(user)', 'Mining-the-Social-Web(repo)'), ('tswicegood(user)', 'Mining-the-Social-Web(repo)'), ('ngpestelos(user)', 'Mining-the-Social-Web(repo)'), ('darron(user)', 'Mining-the-Social-Web(repo)'), ('brunojm(user)', 'Mining-the-Social-Web(repo)'), ('rgaidot(user)', 'Mining-the-Social-Web(repo)'), ('openw

## Example 7. Calculating degree, betweenness, and closeness centrality measures on the Krackhardt kite graph

In [17]:
from operator import itemgetter
from IPython.display import HTML
from IPython.core.display import display

display(HTML('<img src="resources/ch07-github/kite-graph.png" width="400px">'))

# The classic Krackhardt kite graph
kkg = nx.generators.small.krackhardt_kite_graph()

print ("Degree Centrality")
print (sorted(nx.degree_centrality(kkg).items(), 
             key=itemgetter(1), reverse=True))
print()

print ("Betweenness Centrality")
print (sorted(nx.betweenness_centrality(kkg).items(), 
             key=itemgetter(1), reverse=True))
print()

print ("Closeness Centrality")
print (sorted(nx.closeness_centrality(kkg).items(), 
             key=itemgetter(1), reverse=True))

Degree Centrality
[(3, 0.6666666666666666), (5, 0.5555555555555556), (6, 0.5555555555555556), (0, 0.4444444444444444), (1, 0.4444444444444444), (2, 0.3333333333333333), (4, 0.3333333333333333), (7, 0.3333333333333333), (8, 0.2222222222222222), (9, 0.1111111111111111)]

Betweenness Centrality
[(7, 0.38888888888888884), (5, 0.23148148148148148), (6, 0.23148148148148148), (8, 0.2222222222222222), (3, 0.10185185185185183), (0, 0.023148148148148143), (1, 0.023148148148148143), (2, 0.0), (4, 0.0), (9, 0.0)]

Closeness Centrality
[(5, 0.6428571428571429), (6, 0.6428571428571429), (3, 0.6), (7, 0.6), (0, 0.5294117647058824), (1, 0.5294117647058824), (2, 0.5), (4, 0.5), (8, 0.42857142857142855), (9, 0.3103448275862069)]


## Example 8. Adding additional interest edges to the graph through the inclusion of "follows" edges

In [19]:
# Add (social) edges from the stargazers' followers. This can take a while 
# because of all of the potential API calls to GitHub. The approximate number
# of requests for followers for each iteration of this loop can be calculated as
# math.ceil(sg.get_followers() / 100.0) per the API returning up to 100 items
# at a time.

import sys

for i, sg in enumerate(stargazers[:5]):
    
    # Add "follows" edges between stargazers in the graph if any relationships exist
    try:
        for follower in sg.get_followers():
            if follower.login + '(user)' in g:
                g.add_edge(follower.login + '(user)', sg.login + '(user)', 
                           type='follows')
    except: #ssl.SSLError
        print >> sys.stderr, "Encountered an error fetching followers for", \
                             sg.login, "Skipping."
        print >> sys.stderr, e

    print ("Processed", i+1, " stargazers. Num nodes/edges in graph", \
          g.number_of_nodes(), "/", g.number_of_edges())
    print ("Rate limit remaining", client.rate_limiting)

Processed 1  stargazers. Num nodes/edges in graph 1197 / 1299
Rate limit remaining (4859, 5000)
Processed 2  stargazers. Num nodes/edges in graph 1197 / 1299
Rate limit remaining (4858, 5000)
Processed 3  stargazers. Num nodes/edges in graph 1197 / 1299
Rate limit remaining (4857, 5000)
Processed 4  stargazers. Num nodes/edges in graph 1197 / 1299
Rate limit remaining (4855, 5000)
Processed 5  stargazers. Num nodes/edges in graph 1197 / 1299
Rate limit remaining (4854, 5000)


## Example 9. Exploring the updated graph's "follows" edges

In [22]:
from operator import itemgetter
from collections import Counter

# Let's see how many social edges we added since last time.
print (nx.info(g))
print()

# The number of "follows" edges is the difference
print (len([e for e in dict(g.edges()).items() if e[1]['type'] == 'follows']))
print()

# The repository owner is possibly one of the more popular users in this graph.
print (len([e 
           for e in dict(g.edges()).items() 
               if e[1]['type'] == 'follows' and e[0][0] == 'ptwobrussell(user)']))
print()

# Let's examine the number of adjacent edges to each node
print (sorted([n for n in dict(g.degree()).items()], key=itemgetter(1), reverse=True)[:10])
print()

# Consider the ratio of incoming and outgoing edges for a couple of users with 
# high node degrees...

# A user who follows many but is not followed back by many.

print (len(g.out_edges('hcilab(user)')))
print (len(g.in_edges('hcilab(user)')))
print()

# A user who is followed by many but does not follow back.

print (len(g.out_edges('ptwobrussell(user)')))
print (len(g.in_edges('ptwobrussell(user)')))
print()

c = Counter([e[0][0] for e in dict(g.edges()).items() if e[1]['type'] == 'follows'])
popular_users = [ (u, f) for (u, f) in c.most_common() if f > 1 ]
print ("Number of popular users", len(popular_users))
print ("Top 10 popular users:", popular_users[:10])

Name: 
Type: DiGraph
Number of nodes: 1197
Number of edges: 1299
Average in degree:   1.0852
Average out degree:   1.0852

103

0

[('Mining-the-Social-Web(repo)', 1196), ('angusshire(user)', 36), ('hammer(user)', 18), ('tswicegood(user)', 12), ('VagrantStory(user)', 9), ('mcroydon(user)', 8), ('hoffmann(user)', 6), ('tzuryby(user)', 6), ('trietptm(user)', 6), ('sebasmagri(user)', 5)]

0
0

1
0

Number of popular users 11
Top 10 popular users: [('angusshire(user)', 35), ('VagrantStory(user)', 8), ('trietptm(user)', 5), ('rohithadassanayake(user)', 4), ('douglas(user)', 3), ('abecciu(user)', 2), ('stonegao(user)', 2), ('matagus(user)', 2), ('gawbul(user)', 2), ('jquattrocchi(user)', 2)]


## Example 10. Snapshotting (pickling) the graph's state to disk

In [23]:
# Save your work by serializing out (pickling) the graph
nx.write_gpickle(g, "gen_files/github.gpickle.1")

# How to restore the graph...
# import networkx as nx
# g = nx.read_gpickle("resources/ch07-github/data/github.gpickle.1")

## Example 11. Applying centrality measures to the interest graph

In [24]:
from operator import itemgetter

# Create a copy of the graph so that we can iteratively mutate the copy
# as needed for experimentation

h = g.copy()

# Remove the seed of the interest graph, which is a supernode, in order
# to get a better idea of the network dynamics

h.remove_node('Mining-the-Social-Web(repo)')

# XXX: Remove any other nodes that appear to be supernodes.
# Filter any other nodes that you can by threshold
# criteria or heuristics from inspection.

# Display the centrality measures for the top 10 nodes


dc = sorted(nx.degree_centrality(h).items(), 
            key=itemgetter(1), reverse=True)

print ("Degree Centrality")
print (dc[:10])
print()

bc = sorted(nx.betweenness_centrality(h).items(), 
            key=itemgetter(1), reverse=True)

print ("Betweenness Centrality")
print (bc[:10])
print()

print ("Closeness Centrality")
cc = sorted(nx.closeness_centrality(h).items(), 
            key=itemgetter(1), reverse=True)
print (cc[:10])

Degree Centrality
[('angusshire(user)', 0.02928870292887029), ('hammer(user)', 0.01422594142259414), ('tswicegood(user)', 0.00920502092050209), ('VagrantStory(user)', 0.0066945606694560665), ('mcroydon(user)', 0.005857740585774058), ('hoffmann(user)', 0.0041841004184100415), ('tzuryby(user)', 0.0041841004184100415), ('trietptm(user)', 0.0041841004184100415), ('sebasmagri(user)', 0.0033472803347280333), ('rohithadassanayake(user)', 0.0033472803347280333)]

Betweenness Centrality
[('tswicegood(user)', 5.606834731537744e-06), ('mcroydon(user)', 2.803417365768872e-06), ('rdempsey(user)', 0.0), ('frac(user)', 0.0), ('prb(user)', 0.0), ('batasrki(user)', 0.0), ('twleung(user)', 0.0), ('kevinchiu(user)', 0.0), ('nikolay(user)', 0.0), ('ngpestelos(user)', 0.0)]

Closeness Centrality
[('hammer(user)', 0.014225941422594143), ('tswicegood(user)', 0.009112040911204091), ('mcroydon(user)', 0.007455306200076075), ('hoffmann(user)', 0.0041841004184100415), ('tzuryby(user)', 0.0041841004184100415), ('

## Example 12. Adding starred repositories to the graph

In [27]:
# Let's add each stargazer's additional starred repos and add edges 
# to find additional interests.

MAX_REPOS = 500

for i, sg in enumerate(stargazers[:5]):
    print (sg.login)
    try:
        for starred in sg.get_starred()[:MAX_REPOS]: # Slice to avoid supernodes
            g.add_node(starred.name + '(repo)', type='repo', lang=starred.language, \
                       owner=starred.owner.login)
            g.add_edge(sg.login + '(user)', starred.name + '(repo)', type='gazes')
    except: #ssl.SSLError:
        print ("Encountered an error fetching starred repos for", sg.login, "Skipping.")

    print ("Processed", i+1, "stargazers' starred repos")
    print ("Num nodes/edges in graph", g.number_of_nodes(), "/", g.number_of_edges())
    print ("Rate limit", client.rate_limiting)

rdempsey
Processed 1 stargazers' starred repos
Num nodes/edges in graph 1695 / 1797
Rate limit (4849, 5000)
frac
Processed 2 stargazers' starred repos
Num nodes/edges in graph 1832 / 1938
Rate limit (4847, 5000)
prb
Processed 3 stargazers' starred repos
Num nodes/edges in graph 2010 / 2122
Rate limit (4845, 5000)
mcroydon
Processed 4 stargazers' starred repos
Num nodes/edges in graph 2497 / 2621
Rate limit (4840, 5000)
batasrki
Processed 5 stargazers' starred repos
Num nodes/edges in graph 2558 / 2690
Rate limit (4839, 5000)


**NOTE: Given that Example 12 is potentially a very time-consuming example to run, be sure to snapshot your work**

In [28]:
# Save your work by serializing out another snapshot of the graph
nx.write_gpickle(g, "gen_files/github.gpickle.2")

#import networkx as nx
#g = nx.read_gpickle("resources/ch07-github/data/github.gpickle.2")

Consider analysis similar to _Example 12_ here. Create a copy of the graph and be selective in pruning it or extracting subgraphs of interst.

## Example 13. Exploring the graph after updates with additional starred repositories

In [34]:
# Poke around: how to get users/repos
from operator import itemgetter

print (nx.info(g))
print()

# Get a list of repositories from the graph.

repos = [n for n in g.node() if g.node[n]['type'] == 'repo']

# Most popular repos

print ("Popular repositories")
print (sorted([(n,d) 
              for (n,d) in g.degree() 
                  if g.node[n]['type'] == 'repo'], \
             key=itemgetter(1), reverse=True)[:10])
print()

# Projects gazed at by a user

print ("Respositories that ptwobrussell has bookmarked")
print ([(n,g.node[n]['lang']) 
       for n in g['ptwobrussell(user)'] 
           if g['ptwobrussell(user)'][n]['type'] == 'gazes'])
print()

# Programming languages for each user

print ("Programming languages ptwobrussell is interested in")
print (list(set([g.node[n]['lang'] 
                for n in g['ptwobrussell(user)'] 
                    if g['ptwobrussell(user)'][n]['type'] == 'gazes'])))
print()

# Find supernodes in the graph by approximating with a high number of 
# outgoing edges

print ("Supernode candidates")
print (sorted([(n, len(g.out_edges(n))) 
              for n in g.node()
                  if g.node[n]['type'] == 'user' and len(g.out_edges(n)) > 500], \
             key=itemgetter(1), reverse=True))

Name: 
Type: DiGraph
Number of nodes: 2558
Number of edges: 2690
Average in degree:   1.0516
Average out degree:   1.0516

Popular repositories
[('Mining-the-Social-Web(repo)', 1196), ('system-design-primer(repo)', 3), ('prophet(repo)', 2), ('Apriori(repo)', 2), ('elasticsearch-dsl-py(repo)', 2), ('sounder(repo)', 2), ('pix2code(repo)', 2), ('sanic(repo)', 2), ('ctop(repo)', 2), ('models(repo)', 2)]

Respositories that ptwobrussell has bookmarked
[('Mining-the-Social-Web(repo)', 'JavaScript')]

Programming languages ptwobrussell is interested in
['JavaScript']

Supernode candidates
[('mcroydon(user)', 501)]


## Example 14. Updating the graph to include nodes for programming languages

In [40]:
# Iterate over all of the repos, and add edges for programming languages 
# for each person in the graph. We'll also add edges back to repos so that 
# we have a good point to "pivot" upon.

repos = [n 
         for n in g.node()
             if g.node[n]['type'] == 'repo']

for repo in repos:
    lang = (g.node[repo]['lang'] or "") + "(lang)"
    
    stargazers = [u 
                  for (u, r, d) in g.in_edge()
                     if d['type'] == 'gazes'
                 ]
    
    for sg in stargazers:
        g.add_node(lang, type='lang')
        g.add_edge(sg, lang, type='programs')
        g.add_edge(lang, repo, type='implements')

AttributeError: 'DiGraph' object has no attribute 'in_edge'

## Example 15. Sample queries for the final graph

In [43]:
# Some stats

print (nx.info(g))
print()

# What languages exist in the graph?

print ([n 
       for n in g.nodes_iter() 
           if g.node[n]['type'] == 'lang'])
print()

# What languages do users program with?
print ([n 
       for n in g['ptwobrussell(user)'] 
           if g['ptwobrussell(user)'][n]['type'] == 'programs'])

# What is the most popular programming language?
print ("Most popular languages")
print (sorted([(n, g.in_degree(n)))
 for n in g.nodes_iter() 
     if g.node[n]['type'] == 'lang'], key=itemgetter(1), reverse=True)[:10]
print()

# How many users program in a particular language?
python_programmers = [u 
                      for (u, l) in g.in_edges_iter('Python(lang)') 
                          if g.node[u]['type'] == 'user']
print ("Number of Python programmers:", len(python_programmers))
print()

javascript_programmers = [u for 
                          (u, l) in g.in_edges_iter('JavaScript(lang)') 
                              if g.node[u]['type'] == 'user']
print ("Number of JavaScript programmers:", len(javascript_programmers))
print()

# What users program in both Python and JavaScript?
print ("Number of programmers who use JavaScript and Python")
print (len(set(python_programmers).intersection(set(javascript_programmers))))

# Programmers who use JavaScript but not Python
print ("Number of programmers who use JavaScript but not Python"
print len(set(javascript_programmers).difference(set(python_programmers)))

# XXX: Can you determine who is the most polyglot programmer?

SyntaxError: invalid syntax (<ipython-input-43-3f21afc52ca7>, line 20)

**NOTE: Optionally, snapshot the final graph**

In [ ]:
# Save your work by serializing out another snapshot of the graph
nx.write_gpickle(g, "resources/ch07-github/data/github.gpickle.3")

#import networkx as nx
#g = nx.read_gpickle("resources/ch07-github/data/github.gpickle.3")

## Example 16. Graph visualization of the social network for the original interest graph

In [ ]:
import os
import json
from IPython.display import IFrame
from IPython.core.display import display
from networkx.readwrite import json_graph

print "Stats on the full graph"
print nx.info(g)
print

# Create a subgraph from a collection of nodes. In this case, the
# collection is all of the users in the original interest graph

mtsw_users = [n for n in g if g.node[n]['type'] == 'user']
h = g.subgraph(mtsw_users)

print "Stats on the extracted subgraph"
print nx.info(h)

# Visualize the social network of all people from the original interest graph.

d = json_graph.node_link_data(h)
json.dump(d, open('resources/ch07-github/force.json', 'w'))


# IPython Notebook can serve files and display them into
# inline frames. Prepend the path with the 'files' prefix.

# A D3 template for displaying the graph data.
viz_file = 'files/resources/ch07-github/force.html'

# Display the D3 visualization.

display(IFrame(viz_file, '100%', '600px'))